<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [3]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [4]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [5]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses values(?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

In [6]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses values(?,?,?,?,?)',recent_sales)

conn.commit()

Select all rows from houses

In [7]:
for row in c.execute('SELECT * FROM houses'):
    print (row)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [8]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('C:/Users/Eugene/Documents/IOD-Data Science/Data/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [9]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [10]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [11]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [12]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [13]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

In [14]:
re = c.execute("SELECT * FROM houses WHERE field1 IN (52, 53)")
re.fetchall()

[]

QUIZ: What would `DELETE FROM houses` do?
Delete records from houses

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [15]:
# ANSWER
results1 = c.execute("SELECT * FROM houses WHERE bdrms = 4")
results1.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [16]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [17]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [20]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('C:/Users/Eugene/Documents/IOD-Data Science/Data/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [21]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [22]:
#ANSWER
result2 = c.execute("SELECT AVG(price), bdrms FROM houses_pandas GROUP BY bdrms")
result2.fetchall()

[(169900.0, 1),
 (280866.6666666667, 2),
 (326403.92, 3),
 (377449.78571428574, 4),
 (699900.0, 5)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [3]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [4]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:/Users/Eugene/Documents/IOD-Data Science/Data/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:/Users/Eugene/Documents/IOD-Data Science/Data/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [25]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [26]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [27]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [28]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 3. Check DataTypes 

In [29]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [30]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [2]:
# Establishing Local DB connection
sqlite_db1 = 'eshop.db.sqlite'
db_connection = sqlite3.connect(sqlite_db1)
c = db_connection.cursor()
# Save these two dataframes as a table in sqlite

orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

NameError: name 'orders' is not defined

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [32]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [33]:
# Select Number of Orders for Each Customer

query1 = """
        SELECT customer_name
               ,COUNT(order_id) AS orders_count
        FROM orders 
        GROUP BY customer_name;
        """

orders_each_cust = sql.read_sql(query1, con=db_connection)
orders_each_cust

,customer_name,orders_count
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


#### 6. Select Number of Customers for Each Country

In [34]:
# Select Number of Customers for Each Country

query6 = """
        SELECT country
               ,COUNT(Distinct customer_name) AS customer_count 
        FROM orders 
        GROUP BY country;
        """

orders_each_country = sql.read_sql(query6, con=db_connection)
orders_each_country

,country,customer_count
0,Austria,125
1,Belgium,66
2,Denmark,28
3,Finland,33
4,France,568
5,Germany,505
6,Ireland,48
7,Italy,351
8,Netherlands,172
9,Norway,37


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [35]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query7a = """
        SELECT  order_id
               ,discount
               ,sales
               ,quantity
        FROM orders_break_down;
        """

orders_breakdown = sql.read_sql(query7a, con=db_connection)
orders_breakdown

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [36]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

query7b = """
        SELECT  order_id
               ,discount
               ,sales
               ,quantity
               ,(sales*quantity)-discount AS total_price
        FROM orders_break_down;
        """

orders_breakdown_tp = sql.read_sql(query7b, con=db_connection)
orders_breakdown_tp

,order_id,discount,sales,quantity,total_price
0,IT-2011-3647632,0.5,45.0,3,134.5
1,ES-2011-4869686,0.0,854.0,7,5978.0
2,ES-2011-4939443,0.0,140.0,3,420.0
3,IT-2011-2942451,0.5,27.0,2,53.5
4,IT-2011-2942451,0.5,17.0,2,33.5
...,...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2,490.0
8043,IT-2014-3715679,0.5,30.0,2,59.5
8044,IT-2014-3715679,0.5,23.0,4,91.5
8045,IT-2014-3715679,0.5,108.0,3,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [37]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query7c = """
        SELECT  order_id
               ,discount
               ,sales
               ,quantity
               ,(sales*quantity)-discount AS total_price
        FROM orders_break_down
        WHERE ((sales*quantity)-discount) > 100;
        """

orders_breakdown_tp_mt_100 = sql.read_sql(query7c, con=db_connection)
orders_breakdown_tp_mt_100

,order_id,discount,sales,quantity,total_price
0,IT-2011-3647632,0.5,45.0,3,134.5
1,ES-2011-4869686,0.0,854.0,7,5978.0
2,ES-2011-4939443,0.0,140.0,3,420.0
3,ES-2011-3848439,0.0,90.0,3,270.0
4,ES-2011-3848439,0.0,207.0,4,828.0
...,...,...,...,...,...
6461,ES-2014-2815584,0.0,261.0,6,1566.0
6462,ES-2014-4785777,0.0,32.0,5,160.0
6463,ES-2014-3458802,0.0,245.0,2,490.0
6464,IT-2014-3715679,0.5,108.0,3,323.5


#### 8. Select All Customer And The Product They Have Bought

In [40]:
# Select All Customer And The Product They Have Bought

query8 = """
        SELECT 
         ord.customer_name
         ,obd.product_name
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        ORDER BY 1
        ;
        """

orders_cust_items = sql.read_sql(query8, con=db_connection)
orders_cust_items

,customer_name,product_name
0,Aaron Bergman,"Apple Office Telephone, Cordless"
1,Aaron Bergman,"Elite Box Cutter, High Speed"
2,Aaron Bergman,"Harbour Creations Round Labels, Laser Printer ..."
3,Aaron Bergman,"Binney & Smith Markers, Fluorescent"
4,Aaron Bergman,"Novimex Shipping Labels, Alphabetical"
...,...,...
8042,Zuschuss Donatelli,"Hewlett Fax Machine, Laser"
8043,Zuschuss Donatelli,"Office Star Executive Leather Armchair, Adjust..."
8044,Zuschuss Donatelli,"Accos Rubber Bands, Metal"
8045,Zuschuss Donatelli,"Memorex Numeric Keypad, Bluetooth"


In [41]:
# Select All Customer And The Product They Have Bought

query81 = """
        SELECT 
         ord.customer_name
         ,GROUP_CONCAT(obd.product_name) AS product_name
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        GROUP BY ord.customer_name;
        ;
        """

orders_cust_items1 = sql.read_sql(query81, con=db_connection)
orders_cust_items1

,customer_name,product_name
0,Aaron Bergman,"Apple Office Telephone, Cordless,Elite Box Cut..."
1,Aaron Hawkins,"Epson Inkjet, Wireless,Xerox Note Cards, Recyc..."
2,Aaron Smayling,"Binney & Smith Pens, Fluorescent,Apple Signal ..."
3,Adam Bellavance,"Sauder Classic Bookcase, Mobile,Panasonic Prin..."
4,Adam Hart,"Cardinal Binder Covers, Economy,Ibico Binder C..."
...,...,...
787,Xylona Preis,"Ikea Classic Bookcase, Mobile,Smead Shelving, ..."
788,Yana Sorensen,"Binney & Smith Sketch Pad, Water Color,Enermax..."
789,Yoseph Carroll,"Hon Round Labels, 5000 Label Set,Smead Folders..."
790,Zuschuss Carroll,"BIC Canvas, Blue,Breville Microwave, Silver,Ca..."


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [67]:
# Select Number of 'Furniture' Orders For Each Country

query9a = """
        SELECT 
         ord.country
         ,COUNT(ord.order_id) AS order_count
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
            AND obd.category = 'Furniture'
        GROUP BY  ord.country
        """

orders_fur_country = sql.read_sql(query9a, con=db_connection)
orders_fur_country

,country,order_count
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299
5,Germany,264
6,Ireland,12
7,Italy,136
8,Netherlands,76
9,Norway,6


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [63]:
# Select Number of 'Furniture' Orders For The Country Denmark

query9b = """
        SELECT 
         ord.country
         ,COUNT(ord.order_id) AS furniture_order_count
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
            AND obd.category = 'Furniture'
        WHERE ord.country = 'Denmark'
        GROUP BY  ord.country
        """

orders_fur_denmark = sql.read_sql(query9b, con=db_connection)
orders_fur_denmark

,country,furniture_order_count
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [49]:
# Select Total Sales With Discount and Without Discount for Each Country

query10 = """
        SELECT 
         ord.country
         ,SUM(obd.sales) AS total_sales
         ,SUM(obd.sales)-SUM(obd.discount) AS total_sales_with_discount
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        GROUP BY ord.country
        """

orders_ts = sql.read_sql(query10, con=db_connection)
orders_ts

,country,total_sales,total_sales_with_discount
0,Austria,79382.0,79382.00
1,Belgium,42320.0,42320.00
2,Denmark,7763.0,7732.50
3,Finland,20702.0,20702.00
4,France,609683.0,609536.00
5,Germany,488681.0,488591.15
6,Ireland,15998.0,15947.70
7,Italy,252742.0,252619.40
8,Netherlands,70313.0,70124.10
9,Norway,20529.0,20529.00


#### 11.A Select Total Quantity, Total Sales for Each Country

In [61]:
# Select Total Quantity, Total Sales for Each Country

query11a = """
        SELECT 
         ord.country
         ,SUM(obd.sales) AS total_sales
         ,SUM(obd.quantity) AS total_quantity
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        GROUP BY ord.country
        """

orders_tq_ts = sql.read_sql(query11a, con=db_connection)
orders_tq_ts

,country,total_sales,total_quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329
5,Germany,488681.0,6179
6,Ireland,15998.0,392
7,Italy,252742.0,3612
8,Netherlands,70313.0,1526
9,Norway,20529.0,261


#### 11.B Select Top 3 Country Based on Sales

In [60]:
# Select Top 3 Country Based on Sales

query11b = """
        SELECT 
         ord.country
         ,SUM(obd.sales) AS total_sales
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        GROUP BY ord.country
        ORDER BY 2 DESC
        LIMIT 3
        """

orders_ts_top3 = sql.read_sql(query11b, con=db_connection)
orders_ts_top3

,country,total_sales
0,France,609683.0
1,Germany,488681.0
2,United Kingdom,420497.0


#### 11.C Select Bottom 3 Country Based On Quantities

In [59]:
# Select Bottom 3 Country Based On Quantities


query11c = """
        SELECT 
         ord.country
         ,SUM(obd.quantity) AS total_quantity
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        GROUP BY ord.country
        ORDER BY 2 ASC
        LIMIT 3
        """

orders_tq_bot3 = sql.read_sql(query11c, con=db_connection)
orders_tq_bot3

,country,total_quantity
0,Finland,201
1,Denmark,204
2,Norway,261


#### 12. Select Average Sales By Categroy For The Country 'France'

In [68]:
# Select Average Sales By Categroy For The Country 'France'


query12 = """
        SELECT 
         ord.country
         ,obd.category
         ,AVG(obd.sales) AS average_sales
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        WHERE ord.country = 'France'
        GROUP BY ord.country
        ,obd.category
        """

orders_as_france = sql.read_sql(query12, con=db_connection)
orders_as_france

,country,category,average_sales
0,France,Furniture,582.314381
1,France,Office Supplies,167.217709
2,France,Technology,595.145078


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [5]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

query13 = """
        
        SELECT 
        ord.country
        ,obd.category
        ,SUM(obd.sales) AS total_sales
        FROM orders ord
        INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
        INNER JOIN ( 
            SELECT 
             ord.country
             ,obd.category
             ,SUM(obd.sales)/COUNT(obd.sales) AS average_sales
            FROM orders ord
            INNER JOIN orders_break_down obd ON obd.order_id = ord.order_id
            GROUP BY ord.country, obd.category
            ORDER BY 2 DESC
            LIMIT 1)tmp ON tmp.country = ord.country
         GROUP BY  ord.country
         ,obd.category
        """

orders_ts_has = sql.read_sql(query13, con=db_connection)
orders_ts_has

,country,category,total_sales
0,Austria,Furniture,21091.0
1,Austria,Office Supplies,31205.0
2,Austria,Technology,27086.0


In [7]:
query14 = """
    SELECT
        orders.country
        ,orders_break_down.category
        ,SUM(sales) AS total_sales
        ,AVG(sales) AS avg_sales       
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country, orders_break_down.category
    ORDER BY avg_sales 
    DESC LIMIT 1
    """

sql.read_sql(query14, con=db_connection)

,country,category,total_sales,avg_sales
0,Switzerland,Technology,11737.0,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



